In [1]:
import cv2
import numpy as np

from pose_vector_to_transformation_matrix import pose_vector_to_transformation_matrix
from project_points import project_points
from undistort_image import undistort_image

In [2]:
# load camera poses
pose_vectors = np.loadtxt('data/all_pose_vectors_1.txt')

# load camera intrinsics
K = np.loadtxt('data/K_1.txt')  # calibration matrix[3x3]
D = np.loadtxt('data/D_1.txt')  # distortion coefficients[2x1]

In [3]:
import open3d as o3d
import numpy as np

# BunnyMesh 데이터 다운로드 및 경로 가져오기
bunny_path = o3d.data.BunnyMesh().path

# 3D 토끼 메쉬 로드
bunny_mesh = o3d.io.read_triangle_mesh(bunny_path)

# 메쉬를 포인트 클라우드로 변환 (표면 샘플링)
bunny_pcd = bunny_mesh.sample_points_poisson_disk(number_of_points=5000)

# 포인트 클라우드의 좌표 가져오기
points = np.asarray(bunny_pcd.points)  # (5000, 3) 형태의 NumPy 배열

# 각 포인트에 1 추가하여 (5000, 4) 배열 생성
points_homogeneous = np.hstack([points, np.ones((points.shape[0], 1))])  # (5000, 4)

# x축 기준 90도 회전 행렬 정의
R_x_90 = np.array([
    [1,  0,  0, 0],
    [0,  0, -1, 0],
    [0,  1,  0, 0],
    [0,  0,  0, 1]
])

R_z_180 = np.array([
    [-1,  0,  0, 0],
    [ 0, -1,  0, 0],
    [ 0,  0,  1, 0],
    [ 0,  0,  0, 1]
])

# 회전 변환 적용
points_homogeneous = points_homogeneous @ R_x_90.T  # 행렬 곱 (Transpose 필요)

points_homogeneous = points_homogeneous @ R_z_180.T

# 결과 확인
print("포인트 클라우드 좌표 (x, y, z, 1):")
print(points_homogeneous)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
포인트 클라우드 좌표 (x, y, z, 1):
[[ 0.08744034  0.01749552  0.11047664  1.        ]
 [ 0.01221762  0.00814562  0.12942696  1.        ]
 [ 0.01885284  0.01075152  0.12788719  1.        ]
 ...
 [ 0.02820382 -0.00456338  0.03832058  1.        ]
 [ 0.04747543  0.04372133  0.03501536  1.        ]
 [ 0.06673022  0.0349141   0.03719474  1.        ]]


In [4]:
# 좌표 정렬
points_homogeneous[:, :3] *= 3000
points_homogeneous[:, 2] *= -1
points_homogeneous[:, 0] += 300
points_homogeneous[:, 1] += 300

In [12]:
from tqdm.notebook import tqdm
from PIL import Image

images = []

for ii in tqdm(range(27)):
    img_index = ii + 1
    img = cv2.imread('data/images/cam1/cam1_{0:04d}.jpg'.format(img_index))

    # undistort image with bilinear interpolation
    img_undistorted = undistort_image(img, K, D, bilinear_interpolation=True)

    # project the corners on the image
    # compute the 4x4 homogeneous transformation matrix that maps points
    # from the world to the camera coordinate frame

    T_C_W = pose_vector_to_transformation_matrix(pose_vectors[ii, :])

    # project on image coordinate
    p_C_sphere = np.matmul(T_C_W[None, :, :], points_homogeneous[:, :, None]).squeeze(-1)
    p_C_sphere = p_C_sphere[:, :3]

    sphere_pts = project_points(p_C_sphere, K, np.zeros([4, 1]))
    sphere_pts_int = sphere_pts.astype(int)
    sphere_pts_int = np.unique(sphere_pts_int, axis=0)


    # draw points
    for x, y in sphere_pts_int:
        for iy in range(-1, 2):
            for ix in range(-1, 2):
                img_undistorted[y+iy, x+ix] = (0, 255, 0)

    images.append(img_undistorted)

Image.fromarray(images[-1])

  0%|          | 0/27 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
import warnings

# Save to video file
fps = 10
height, width, _ = images[10].shape
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
video_filename = 'ar_wireframe_bunny.avi'
video_writer = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))

with warnings.catch_warnings(record=True):
    for image in tqdm(images):
        video_writer.write(image)

video_writer.release()

  0%|          | 0/27 [00:00<?, ?it/s]

In [10]:
from matplotlib import animation
from IPython.display import display, HTML

import matplotlib.pyplot as plt

def plot_sequence_images(image_array):
    ''' Display images sequence as an animation in jupyter notebook

    Args:
        image_array(numpy.ndarray): image_array.shape equal to (num_images, height, width, num_channels)
    '''
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[:2]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array), interval=33, repeat_delay=3, repeat=True)
    display(HTML(anim.to_html5_video()))

In [11]:
plot_sequence_images(images)

<Figure size 1600x900 with 0 Axes>